# מעבדה 3 - כיול מצלמה  - בניית נתונים

עד כה יצרנו תצלומים באמצעות מצלמות אידאליות, שם הנקודה הראשית נמצאת בדיוק בראשית הצירים של מערכת התצלום.

על מנת שנוכל לבדוק מודלים של כיול של מצלמה, עלינו להכניס הזזה של הנקודה הראשית ועיוותי עדשה.

### חלק 1 - הנקודה הראשית

1.	כתבו פונקציה המייצרת נקודות במערכת התצלום כאשר הנקודה הראשית מוזזת מראשית הצירים. 

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from Camera import Camera
import PhotoViewer as pv
from SingleImage import SingleImage
import MatrixMethods

# def ShiftedPrincipalPoint(self, camera_points):
#     """
#     Points in camera space when principal point is shifted
#     :param camera_points: sampled image points
#     :param t: shifting from principal point

#     :type camera_points: np.array nx2
#     :type t: np.array 1X2

#     :return: shifted image points
#     """

#     shifted_camera_points = np.zeros([len(camera_points), 2])
#     shifted_camera_points[:, 0] = camera_points[:, 0] - self.principalPoint[0]
#     shifted_camera_points[:, 1] = camera_points[:, 1] - self.principalPoint[1]
#     return shifted_camera_points


2.	נגדיר את מיקום הנקודה הראשית xp=100; yp=200 מיקרון. כיצד ישתנו ערכי הנקודות במערכת התצלום. הסבירו איזה ערכים אתם צופים לקבל. באיזו סוג התמרה השתמשתם?

ביצענו הזזה של מערכת היחוס.
נחסר את ערכי ההזזה מערכי הנקודות שדגמנו במערכת התצלום.


In [7]:
# define camera
focal_length = 35  # [mm]
sensor_size = 3500  # [micron]
xp=100
yp=200
camera1 = Camera(focal_length, np.array([xp, yp]), None, None, None, sensor_size)

camera_points = np.array([[2000, 2000],
                             [1500, 400],
                              [1000, 800],
                              [2300, 2100],
                              [1300, 1300],
                              [700, 700]])
print(camera_points)

[[2000 2000]
 [1500  400]
 [1000  800]
 [2300 2100]
 [1300 1300]
 [ 700  700]]


3.	הראו שהפונקציה מחזירה את הערכים שלהם ציפיתם. 

In [8]:
shifted_points = camera1.ShiftedPrincipalPoint(camera_points)
print('~~~~~','\n',shifted_points)


~~~~~ 
 [[1900. 1800.]
 [1400.  200.]
 [ 900.  600.]
 [2200. 1900.]
 [1200. 1100.]
 [ 600.  500.]]


4.	כתבו פונקציה המתקנת את הנקודות בהינתן ערכי הנקודה הראשית אל מערכת המצלמה. 

In [4]:
# def CorrectionToPrincipalPoint(self, camera_points):
#     """
#     Correction to principal point

#     :param camera_points: sampled image points

#     :type: np.array nx2

#     :return: corrected image points

#     :rtype: np.array nx2

#     .. warning::

#         This function is empty, need implementation

#     .. note::

#         The principal point is an attribute of the camera object, i.e., ``self.principalPoint``


#     """

#     corrected_camera_points = np.zeros([len(camera_points), 2])
#     corrected_camera_points[:, 0] = camera_points[:, 0] + self.principalPoint[0]
#     corrected_camera_points[:, 1] = camera_points[:, 1] + self.principalPoint[1]
#     return corrected_camera_points


5.	הראו כי פתרון התצלום לאחר התיקון מוביל לערכים צפויים. 

In [9]:
correted_points = camera1.CorrectionToPrincipalPoint(shifted_points)
print('~~~~~','\n',correted_points)


~~~~~ 
 [[2000. 2000.]
 [1500.  400.]
 [1000.  800.]
 [2300. 2100.]
 [1300. 1300.]
 [ 700.  700.]]


ניתן לראות שחזרנו לנקודות המקוריות- כלומר התיקון עבד כראוי

### חלק 2 - עיוותים רדיאליים

6.	השתמשו במודל של העיוותים הרדיאליים שניתנה בהרצאה. מה יהיו ערכי K1  ו-K2 כך שהעיוות המקסימלי בתמונה לא יעלה על 50 מיקרון והעיוות הנתרם על ידי K2 לא יעלה על 10 מיקרון. ניתן להניח ש-K3=0.  

נקבע את-
k2
ואז נחפש את- 
k1

7.	הסתמכו על מודל העיוותים ועל K1 ו-K2 שמצאתם בסעיף הקודם, ובנו גרף המתאר את גודל העיוות הרדיאלי כתלות במרחק מהנקודה הראשית.

8.	כתבו פונקציה המוסיפה את העיוותים הרדיאליים (K1  ו-K2 ) של העדשה לנקודות במערכת התצלום. 

9.	השתמשו בערכי K1 ו-K2 שמצאתם בסעיף 6. הראו כיצד ישפיע מיקום שונה של הנקודה הראשית על גודל העיוות המדידות. 

10.	שרטטו את העיוותים. השתמשו ב- ()plt.quiver. 

11.	האם גודל העיוות הרדיאלי תלוי בפיזור נקודות הבקרה? נמקו.

In [ ]:
פיזור בתצלום

12.	האם גודל העיוות הרדיאלי תלוי באוריינטציית התצלום? נמקו.